In [1]:
import sys
import random
import pickle
from os.path import join

sys.path.append('..')

import numpy as np
import networkx as nx

from cnrg.VRG import VRG as VRG
from cnrg.LightMultiGraph import convert
from cnrg.LightMultiGraph import LightMultiGraph as LightMultiGraph

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
sys.path.append('../src')
from utils import silence
from data import load_data
# from data_old import read_data
from decomposition import decompose, decompose_component
from graph_transitions import update_grammar
from utils import silence, find, replace
from utils import graph_edit_distance as ged

In [3]:
loaded = load_data(dataname='email-eucore', lookback=0)
graphs = [g for _, g in loaded]

In [4]:
grammar = decompose(graphs[0], time=0)

100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]
100%|███████████████████████████████|[00:00<00:00]


In [9]:
joint_grammar = update_grammar(grammar, graphs[0], graphs[1], 1, mode='joint')
print(joint_grammar.ll())

additions: 100%|██████████| 2207/2207 [00:02<00:00, 784.30it/s]

graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 241(240) mdl: 26_931.7 bits
-7.716015266642587


In [10]:
indep_grammar = update_grammar(grammar, graphs[0], graphs[1], 1, mode='indep')
print(indep_grammar.ll())

additions: 100%|██████████| 2289/2289 [00:01<00:00, 1455.83it/s]

graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 232(231) mdl: 26_034.7 bits
-7.734121303328305


In [5]:
n = graphs[1].order()
p = (2 * graphs[1].size()) / (n * (n - 1))
er = nx.fast_gnp_random_graph(graphs[1].order(), p)

In [6]:
print(graphs[1].order(), graphs[1].size())
print(er.order(), er.size())

702 3463
702 3472


In [11]:
erjoint = update_grammar(grammar, graphs[0], er, 1, mode='joint')
print(erjoint.ll())

additions: 100%|██████████| 1699/1699 [00:01<00:00, 904.25it/s]

graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 358(357) mdl: 38_217.3 bits
-7.754481547470383


In [12]:
erindep = update_grammar(grammar, graphs[0], er, 1, mode='indep')
print(erindep.ll())

additions: 100%|██████████| 3159/3159 [00:13<00:00, 231.28it/s] 


is_isomorphic interrupted after 10 seconds running on [9 -> (22, 23)[3], 9 -> (22, 22)[3]]


additions: 100%|██████████| 298/298 [00:00<00:00, 1458.68it/s]

graph: , mu: 4, type: mu_level_dl clustering: leiden rules: 232(231) mdl: 74_825.9 bits
-8.145259566516865


In [ ]:
grammar.rule_dict[0][2].graph.edges(data=True)

In [ ]:
h = nx.Graph()
h.add_nodes_from([0, 1, 2, 3, 4, 5])
h.add_edges_from([(i, i + 1) for i in range(5)])
hgrammar = decompose(h)
hgrammarc = decompose_component(h)

In [ ]:
hgrammar.rule_list

In [ ]:
for rule in hgrammarc.rule_list:
    print(f'{rule.lhs} -> ', end='')
    labels = nx.get_node_attributes(rule.graph, 'label')  # | nx.get_node_attributes(rule.graph, 'b_deg')
    nx.draw(rule.graph, labels=labels)
    plt.show()

In [ ]:
grammar.rule_dict[0][1].graph.nodes(data=True)

In [ ]:
for s in nx.connected_components(graphs[0]):
    print(graphs[0].subgraph(s).size())

In [ ]:
newedgesizes = [rule.graph.size() for rule in new_grammar.rule_list]
display(sorted(edgesizes))

In [ ]:
for rule in new_grammar.rule_list:
    if rule.edit_cost != 0:
        print(rule.edit_cost)

In [ ]:
len(new_grammar.rule_list) - len(grammar.rule_list)

In [ ]:
for rule in new_grammar.rule_list:
    print(rule.edit_cost)

In [ ]:
for rule in new_grammar.rule_list:
    print(rule.time)

In [ ]:
r0 = grammar.rule_list[0]
r0.time

In [ ]:
g0 = grammar.rule_list[0].graph
print(g0.order(), g0.size())

In [ ]:
grammar.rule_list[6].graph.nodes(data=True)

In [ ]:
def node_match(d1, d2):
    if d1.keys() != d2.keys():
        return False
    
    if 'label' in d1 and d1['label'] != d2['label']:
        return False
    
    if 'b_deg' in d1 and d1['b_deg'] != d2['b_deg']:
        return False
    
    return True

In [ ]:
%%time
edit_dists = [nx.graph_edit_distance(g0, rule.graph, node_match=node_match, timeout=None)
              for rule in grammar.rule_list]

In [ ]:
edit_dists

In [ ]:
%%time
opt_edit_dists = []
for rule in grammar.rule_list:
    g = rule.graph
    for val in nx.optimize_graph_edit_distance(g0, g, node_match=node_match):
        # print('.', end='')
        minval = val
    opt_edit_dists += [minval]
    # print(f'min: {minval}')

In [ ]:
edit_dists == opt_edit_dists

# independent sequential experiments

In [ ]:
dataname = 'email-eucore'
# facebook-links ['weight'] edge missing?
lookbacks = [0]

base_grammars = {lookback: [] for lookback in lookbacks}
joint_grammars = {lookback: [] for lookback in lookbacks}
indep_grammars = {lookback: [] for lookback in lookbacks}

base_mdls = {lookback: [] for lookback in lookbacks}
joint_mdls = {lookback: [] for lookback in lookbacks}
joint_lls = {lookback: [] for lookback in lookbacks}
indep_mdls = {lookback: [] for lookback in lookbacks}
indep_lls = {lookback: [] for lookback in lookbacks}

mu = 4
for lookback in lookbacks:
    loaded = load_data(dataname=dataname, lookback=lookback)
    graphs = [g for _, g in loaded]
    years = [t for t, _ in loaded][:len(graphs) - 1]
    
    # graphs, years = read_data(dataname=dataname, lookback=lookback)
    
    graphs = graphs[0:]
    
    base_mdl = []
    joint_ll = []
    joint_mdl = []
    indep_ll = []
    indep_mdl = []

    for idx, (home_graph, away_graph) in enumerate(zip(graphs[:-1], graphs[1:])):
        base_grammar = decompose(home_graph, mu=mu)
        joint_grammar = update_grammar(base_grammar, home_graph, away_graph, mode='joint')
        indep_grammar = update_grammar(base_grammar, home_graph, away_graph, mode='independent')
        
        base_grammars[lookback] += [base_grammar]
        joint_grammars[lookback] += [joint_grammar]
        indep_grammars[lookback] += [indep_grammar]
        
        joint_mdl += [joint_grammar.calculate_cost()]
        joint_ll += [joint_grammar.conditional_ll()]
        
        joint_mdls[lookback] += [joint_grammar.calculate_cost()]
        joint_lls[lookback] += [joint_grammar.conditional_ll()]
        indep_mdls[lookback] += [indep_grammar.calculate_cost()]
        indep_lls[lookback] += [indep_grammar.conditional_ll()]
        
        base_mdls[lookback] += [base_grammar.calculate_cost()]
    
    base_grammar = decompose(graphs[-1], mu=mu)
    base_grammars[lookback] += [base_grammar]
    base_mdls[lookback] += [base_grammar.calculate_cost()]

    with open(f'../results/experiment_sequential/{dataname}_base.grammars', 'wb') as outfile:
        pickle.dump(base_grammars, outfile)
    with open(f'../results/experiment_sequential/{dataname}_joint.grammars', 'wb') as outfile:
        pickle.dump(joint_grammars, outfile)
    with open(f'../results/experiment_sequential/{dataname}_indep.grammars', 'wb') as outfile:
        pickle.dump(indep_grammars, outfile)

    with open(f'../results/experiment_sequential/{dataname}_base.mdls', 'wb') as outfile:
        pickle.dump(base_mdls, outfile)
    with open(f'../results/experiment_sequential/{dataname}_joint.mdls', 'wb') as outfile:
        pickle.dump(joint_mdls, outfile)
    with open(f'../results/experiment_sequential/{dataname}_joint.lls', 'wb') as outfile:
        pickle.dump(joint_lls, outfile)
    with open(f'../results/experiment_sequential/{dataname}_indep.mdls', 'wb') as outfile:
        pickle.dump(indep_mdls, outfile)
    with open(f'../results/experiment_sequential/{dataname}_indep.lls', 'wb') as outfile:
        pickle.dump(indep_lls, outfile)

In [ ]:
print('hi')

In [ ]:
loaded = load_data(dataname='email-dnc', lookback=0)
graphs = [g for _, g in loaded]
[(g.order(), g.size()) for g in graphs]

In [ ]:
loaded = load_data(dataname='facebook-links', lookback=0)
graphs = [g for _, g in loaded]
[(g.order(), g.size()) for g in graphs]
#[(len(set(cur.nodes()) & set(nxt.nodes())), len(set(cur.edges()) & set(nxt.edges()))) for cur, nxt in zip(graphs[:-1], graphs[1:])]

In [ ]:
graphs, _ = read_data(dataname='fb-messages', lookback=0)
[(len(set(cur.nodes()) & set(nxt.nodes())), len(set(cur.edges()) & set(nxt.edges()))) for cur, nxt in zip(graphs[:-1], graphs[1:])]

In [ ]:
joint_lls

In [ ]:
indep_lls

In [ ]:
#with plt.style.context(['ipynb', 'use_mathtext', 'colors5-light']):
plt.title(f'sequential experiments: {dataname}, lookback {lookback}')
plt.xlabel('time')
plt.ylabel('log likelihood')
plt.plot(range(len(years)), joint_lls[0], label='joint model')
plt.plot(range(len(years)), indep_lls[0], label='independent model')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.ticklabel_format(style='plain')
    #plt.savefig(f'../figures/exp3_{dataname}_{cumulative}.svg')

In [ ]:
#with plt.style.context(['ipynb', 'use_mathtext', 'colors5-light']):
plt.title(f'sequential experiments: {dataname}, lookback {lookback}')
plt.xlabel('time')
plt.ylabel('minimal description length')
plt.plot(range(len(years)), joint_mdls[0], label='joint model')
plt.plot(range(len(years)), indep_mdls[0], label='independent model')
plt.plot(range(len(years)), [base_mdls[0] for _ in years], label='base model')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.ticklabel_format(style='plain')
    #plt.savefig(f'../figures/exp3_{dataname}_{cumulative}.svg')

In [ ]:
base_mdls

# accumulated sequential experiments